## Automatizar extracción info PDF desde Glide app

### Installing the required libraries

In [1]:
import os
import requests
import pdfplumber
import pandas as pd
import re
import datetime
import io
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Connecting to Google Sheets with OAuth

In [7]:
scope = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("glide-automation-353df74de187.json",scope)
client = gspread.authorize(credentials)
sheet = client.open("DocuHub")

### Getting the value from the selected cell and worksheet

In [11]:
worksheet = sheet.worksheet("Upload_Facturas")
cell_value = worksheet.acell('B2').value
print(cell_value)

https://storage.googleapis.com/glide-prod.appspot.com/uploads-v2/IEX87OO7Al4y6ZKmmJkG/pub/8uWiKKwBbsLiT1MHVgKk.pdf


### Reading a pdf file without downloading in local 

In [57]:
url = 'https://storage.googleapis.com/glide-prod.appspot.com/uploads-v2/IEX87OO7Al4y6ZKmmJkG/pub/8uWiKKwBbsLiT1MHVgKk.pdf'
response = requests.get(url)

In [58]:
with io.BytesIO(response.content) as open_pdf_file:
    read_pdf = pdfplumber.open(open_pdf_file)
    first_page = pdf.pages[0]
    text = first_page.extract_text()

In [59]:
print(text)

Razón Social: Ana María García Martínez
NIF / CIF: 00654715C
CUPS: ES0022000006297223FV
Dir. Suministro: EMBAJADORES 152, 2º, 6, 28045 MADRID (MADRID)
Contrato Acceso: 001000088135 Tarifa: 2.0DHA
Empresa Distribuidora: UFD DISTRIBUCION ELECTRICIDAD, S.A.
Datos Factura Ana María García Martínez
Fecha de Factura: 25/05/2020 GAZTAMBIDE 70, 3º, C
Periodo Facturación: 20/04/2020 - 20/05/2020 (30 días)
28003 MADRID (MADRID)
Factura Nº: 2020052506693
Nº de Contrato: CO-2018-076542_4.3
Fecha fin del contrato de suministro: 08/11/2020  (renovación anual automática)
Importe Total
Término de energía variable
Precio Peaje Precio Coste Energía Precio Total Consumo Total
P1:  0,062012€/kWh     +           0   , 0  8  7   2 00 €/kWh=         0   , 1  4  9  2   1  2   € /kWh          x                   42,00kWh =                   6   , 2  7    €  8,76 €
P2:  0,002215€/kWh     +           0   , 0  7  5   7 02 €/kWh=         0   , 0  7  7  9   1  7   € /kWh          x                   32,00kWh =     

### Downloading the file from the given URL

In [2]:
#def download_file(url):
    #local_filename = url.split('/')[-1]
    
    #with requests.get(url) as r:
        #with open(local_filename, 'wb') as f:
            #f.write(r.content)
            
    #return local_filename        

In [3]:
#document_url = 'https://storage.googleapis.com/glide-prod.appspot.com/uploads-v2/IEX87OO7Al4y6ZKmmJkG/pub/8uWiKKwBbsLiT1MHVgKk.pdf'

In [4]:
#document = download_file(document_url)

In [5]:
#with pdfplumber.open(document) as pdf:
    #first_page = pdf.pages[0]
    #text = first_page.extract_text()

In [60]:
#print(text)

### Los campos que nos interesaría volcar a nuestra base de datos serían:

- Timestamp (fecha en la que el usuario sube la factura al sistema)
- Nº factura
- Proveedor
- Fecha facturación
- Periodo de facturación
- Importe total

### Rules for extract info

In [54]:
# Fecha creación

In [30]:
timestamp = datetime.date.today().strftime('%d-%m-%Y')
timestamp_clean = re.sub('-','/',timestamp)

In [31]:
table_info = {'Timestamp': timestamp_clean}
print(table_info)

{'Timestamp': '03/10/2020'}


In [32]:
table_info['Timestamp'] = timestamp_clean

In [33]:
# Número factura

In [34]:
for row in text.split('\n'):
    if row.startswith('Factura Nº:'):
        factura_n = row.split()[2:]
        numero_factura = re.sub('\D+','',str(factura_n))

In [35]:
numero_factura

'2020052506693'

In [36]:
table_info = {'Número factura': numero_factura}
print(table_info)

{'Número factura': '2020052506693'}


In [37]:
table_info['Número factura'] = numero_factura

In [38]:
# Fecha facturación

In [39]:
for row in text.split('\n'):
    if row.startswith('Fecha de Factura:'):
        fecha = row.split()[3:]
        fecha_clean = fecha[0]
        fecha_facturacion = re.sub('\W+','/',fecha_clean)

In [40]:
fecha_facturacion

'25/05/2020'

In [41]:
table_info = {'Fecha facturación': fecha_facturacion}
print(table_info)

{'Fecha facturación': '25/05/2020'}


In [42]:
table_info['Fecha facturación'] = fecha_facturacion

In [43]:
# Periodo facturación

In [44]:
for row in text.split('\n'):
    if row.startswith('Periodo Facturación:'):
        periodo = row.split()[2:]
        periodo_clean = " ".join(map(str,periodo))
        periodo_facturacion = ' '.join(periodo_clean.split(' ')[:-2])

In [45]:
periodo_facturacion

'20/04/2020 - 20/05/2020'

In [46]:
table_info['Periodo de facturación'] = periodo_facturacion

In [47]:
# Importe total

In [48]:
for row in text.split('\n'):
    if row.startswith('TOTAL FACTURA'):
        total_factura = row.split()[-1]

In [49]:
table_info['Importe total'] = total_factura

In [53]:
table_info

{'Fecha facturación': '25/05/2020',
 'Periodo de facturación': '20/04/2020 - 20/05/2020',
 'Importe total': '25,80€'}

### Converting all the fields into a pandas dataframe

In [207]:
table_info

df = pd.DataFrame(table_info, index=[1]).T

print(df)

                                              1
Fecha facturación                    25/05/2020
Periodo de facturación  20/04/2020 - 20/05/2020
Importe total                            25,80€


### Send information back to Google Drive file